In [1]:
!nvidia-smi

Thu Sep  5 20:04:48 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.28.03              Driver Version: 560.28.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA TITAN RTX               Off |   00000000:1A:00.0 Off |                  N/A |
| 38%   42C    P0             66W /  280W |       1MiB /  24576MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
# fix numpy in colab
import numpy
from IPython.display import clear_output
import os, sys

if not os.path.exists("Mixtral-8x7B-Instruct-v0.1-offloading-demo"):
    !huggingface-cli download lavawolfiee/Mixtral-8x7B-Instruct-v0.1-offloading-demo --quiet --cache-dir /tmp --local-dir Mixtral-8x7B-Instruct-v0.1-offloading-demo


In [3]:
if not os.path.exists("Mixtral-8x7B-Instruct-v0.1"):
    !huggingface-cli download mistralai/Mixtral-8x7B-Instruct-v0.1 --quiet --cache-dir /tmp --local-dir Mixtral-8x7B-Instruct-v0.1-offloading-full-demo

/scratch/abhimanyu/work/Google-MoE/mixtral-offloading-residency-info/notebooks/Mixtral-8x7B-Instruct-v0.1-offloading-full-demo


In [4]:

import os, sys
script_dir = os.getcwd()
module_path = script_dir
for _ in range(1):
    module_path = os.path.abspath(os.path.join(module_path, '../'))
    if module_path not in sys.path:
        sys.path.insert(0,module_path)
        
import torch
from torch.nn import functional as F
from hqq.core.quantize import BaseQuantizeConfig
from huggingface_hub import snapshot_download
from IPython.display import clear_output
from tqdm.auto import trange
from transformers import AutoConfig, AutoTokenizer
from transformers.utils import logging as hf_logging
import time
import gc
from src.build_model import OffloadConfig, QuantConfig, build_model

hqq_aten package not installed. HQQBackend.ATEN backend will not work unless you install the hqq_aten lib in hqq/kernels.


/home/abhimanyu/miniconda3/envs/moe_offload/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# This will reload the imported modules (e.g. get_decode_model_characterstics) every time you execute the jupyter cells, so that you don't need to restart the notebook after updating the source codes.
%load_ext autoreload
%autoreload 2  

In [6]:
full = True

In [7]:
model_name = "mistralai/Mixtral-8x7B-Instruct-v0.1"
if not full:
    quantized_model_name = "lavawolfiee/Mixtral-8x7B-Instruct-v0.1-offloading-demo"
    state_path = "Mixtral-8x7B-Instruct-v0.1-offloading-demo"
    config = AutoConfig.from_pretrained(quantized_model_name)
else:
    state_path = "Mixtral-8x7B-Instruct-v0.1-offloading-full-demo"
    config = AutoConfig.from_pretrained(model_name)


device = torch.device("cuda:0")

##### Change this to 5 if you have only 12 GB of GPU VRAM #####
offload_per_layer = 6
# offload_per_layer = 5
###############################################################

num_experts = config.num_local_experts

offload_config = OffloadConfig(
    main_size=config.num_hidden_layers * (num_experts - offload_per_layer),
    offload_size=config.num_hidden_layers * offload_per_layer,
    buffer_size=2,
    offload_per_layer=offload_per_layer,
)


attn_config = BaseQuantizeConfig(
    nbits=4,
    group_size=64,
    quant_zero=True,
    quant_scale=True,
)
attn_config["scale_quant_params"]["group_size"] = 256


ffn_config = BaseQuantizeConfig(
    nbits=2,
    group_size=16,
    quant_zero=True,
    quant_scale=True,
)
quant_config = QuantConfig(ffn_config=ffn_config, attn_config=attn_config)


# del model

gc.collect
torch.cuda.empty_cache()


## DEFAULT with 4 Experts on GPU

In [8]:
if 'model' in locals():
    del model
model = build_model(
    device=device,
    quant_config=quant_config if not full else None,
    offload_config=offload_config,
    state_path=state_path,
    routing_strategy="TOP-K",
    routing_threshold=0.10
)

/home/abhimanyu/miniconda3/envs/moe_offload/lib/python3.11/site-packages/torch/nn/init.py:453: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


OutOfMemoryError: CUDA out of memory. Tried to allocate 112.00 MiB. GPU 0 has a total capacity of 23.46 GiB of which 34.81 MiB is free. Including non-PyTorch memory, this process has 23.43 GiB memory in use. Of the allocated memory 23.09 GiB is allocated by PyTorch, and 156.88 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
from transformers import TextStreamer


tokenizer = AutoTokenizer.from_pretrained(model_name)
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)
past_key_values = None
sequence = None

seq_len = 0
# while True:
user_input = "Where is Georgia Tech? What is the name of its mascot?"

user_entry = dict(role="user", content=user_input)
input_ids = tokenizer.apply_chat_template([user_entry], return_tensors="pt").to(model.device)

if past_key_values is None:
  attention_mask = torch.ones_like(input_ids)
else:
  seq_len = input_ids.size(1) + past_key_values[0][0][0].size(1)
  attention_mask = torch.ones([1, seq_len - 1], dtype=torch.int, device=model.device)



# sequence = result["sequences"]
# past_key_values = result["past_key_values"]

In [ ]:
start_time = time.time()
result = model.generate(
  input_ids=input_ids,
  attention_mask=attention_mask,
  past_key_values=past_key_values,
  streamer=streamer,
  do_sample=True,
  temperature=0.9,
  top_p=0.9,
  min_new_tokens=20,
  max_new_tokens=20,
  pad_token_id=tokenizer.eos_token_id,
  return_dict_in_generate=True,
  output_hidden_states=False,
  output_router_logits=False,
  output_scores=True, 
)
latency = time.time() - start_time
# total_experts_saved = 0
# for i in result['router_logits'][-32:]:
#     total_experts_saved += i[1]
# total_experts_saved 

Num_tokens = result['sequences'].shape[1] - input_ids.shape[1]
print(f"Total Latency :{latency} sec, Throughput:{Num_tokens/latency}")

In [ ]:
result.keys()